In [ ]:
Hola Jonis , empieza aqui

la tarea esta en este link https://docs.google.com/document/d/1_EqTWLKm_eWDj24cHRPUbb_zAB6Rz-WKWc8IC7coYwo/edit?usp=sharing



ES PARA HOY A LAS 12 PM

In [18]:
!git clone https://github.com/fmplaza/EmoEvent.git

fatal: destination path 'EmoEvent' already exists and is not an empty directory.


In [20]:
import pandas as pd
df = pd.read_csv("/content/EmoEvent/emoevent_es.csv", sep="\t")  # Ajusta el nombre y separador según el archivo
print(df.head())

   id                                              tweet  emotion  offensive
0   1  Acabo de ver la gran pérdida que estamos tenie...  sadness          0
1   2  USER ¿A que vamos a reconstruir Notre Dame ant...  sadness          0
2   3  Desde ayer andan sufriendo por el incendio de ...    anger          0
3   4  Muy afectada -como muchísima gente- por lo ocu...  sadness          0
4   5  Es una mierda lo que paso en HASHTAG pero plea...  disgust          1


## Modelo 1: Red Neuronal (LSTM)

In [ ]:
# ============================================
# Clasificación de tweets con LSTM (Keras/TensorFlow) — Rápido (5–10 min)
# Dataset esperado: df[id, tweet, emotion, offensive]
# Por defecto entrena 'emotion' (multiclase). Cambia TARGET_COL a 'offensive' (binario).
# ============================================

# ---------- Instalar dependencias ----------
!pip -q install -U tensorflow scikit-learn pandas numpy

# ---------- Imports y setup ----------
import os, random, numpy as np, pandas as pd, tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support

print("TensorFlow:", tf.__version__)

# Monta Google Drive (opcional) para guardar el modelo
SAVE_BASE = "/content"
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    SAVE_BASE = "/content/drive/MyDrive/lstm_tweets_colab"
except:
    pass
os.makedirs(SAVE_BASE, exist_ok=True)

# Reproducibilidad
SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

# Limitar uso de memoria GPU (opcional pero recomendado en Colab)
try:
    gpus = tf.config.list_physical_devices('GPU')
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print("GPU:", gpus)
except Exception as e:
    print("No se configuró memory growth:", e)

# ---------- CARGA TU DATA ----------
# Opción A) CSV (descomenta y ajusta la ruta):
# df = pd.read_csv("/content/drive/MyDrive/tu_dataset.csv")

# Opción B) Ejemplo mínimo (borra si usas CSV)
df = pd.DataFrame({
    "id": [1,2,3,4,5]*300,  # simula 1500 filas
    "tweet": [
        "Acabo de ver la gran pérdida que estamos teniendo...",
        "USER ¿A que vamos a reconstruir Notre Dame antes que...",
        "Desde ayer andan sufriendo por el incendio de ...",
        "Muy afectada por lo ocurrido...",
        "Es una mierda lo que pasó en HASHTAG..."
    ]*300,
    "emotion": ["sadness","sadness","anger","sadness","disgust"]*300,
    "offensive": [0,0,0,0,1]*300
})

required = {"id","tweet","emotion","offensive"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"Faltan columnas en df: {missing}")

df = df.dropna(subset=["tweet"]).reset_index(drop=True)
df["tweet"] = df["tweet"].astype(str).str.strip()
print("Total filas:", len(df))

# ---------- Parámetros rápidos (ajusta para 5–10 min) ----------
TARGET_COL   = "emotion"   # "emotion" (multiclase) o "offensive" (binario)
TEXT_COL     = "tweet"

N_SUBSAMPLE  = 1000        # submuestreo para acelerar (baja/sube según tu tiempo)
MAX_TOKENS   = 20000       # vocab size de TextVectorization
MAX_LEN      = 64          # longitud de secuencia (tweets cortos)
EMB_DIM      = 128         # tamaño de embedding
LSTM_UNITS   = 128         # unidades de LSTM (BiLSTM)
DROPOUT      = 0.3
BATCH_SIZE   = 64          # si te da OOM baja a 32
EPOCHS       = 1           # 1 época para prueba rápida
LR           = 2e-3

if len(df) > N_SUBSAMPLE:
    df = df.sample(n=N_SUBSAMPLE, random_state=SEED).reset_index(drop=True)
print("Filas tras submuestreo:", len(df))

# ---------- Etiquetas ----------
if TARGET_COL == "emotion":
    le = LabelEncoder()
    y = le.fit_transform(df[TARGET_COL])
    num_classes = len(le.classes_)
    is_binary = False
    print("Clases:", list(le.classes_))
else:
    y = df[TARGET_COL].astype(int).values
    num_classes = 2
    is_binary = True
    print("Clases: [0, 1] (not_offensive, offensive)")

X_text = df[TEXT_COL].values

# ---------- Split train/valid ----------
X_train, X_val, y_train, y_val = train_test_split(
    X_text, y, test_size=0.15, random_state=SEED, stratify=y
)

# ---------- Capa de vectorización ----------
from tensorflow.keras.layers import TextVectorization

# Normalización básica para tweets (minúsculas, elimina URLs/handles opcionalmente)
def custom_standardize(x):
    x = tf.strings.lower(x)
    # quita URLs y @usuario (opcional)
    x = tf.strings.regex_replace(x, r"http\S+", " ")
    x = tf.strings.regex_replace(x, r"@\w+", " ")
    x = tf.strings.regex_replace(x, r"#", " ")
    # colapsa espacios
    x = tf.strings.regex_replace(x, r"\s+", " ")
    return tf.strings.strip(x)

vectorizer = TextVectorization(
    standardize=custom_standardize,
    max_tokens=MAX_TOKENS,
    output_mode="int",
    output_sequence_length=MAX_LEN
)

# Ajusta vocab con entrenamiento
text_ds = tf.data.Dataset.from_tensor_slices(X_train).batch(256)
vectorizer.adapt(text_ds)

# ---------- Datasets tf.data ----------
def make_ds(texts, labels, batch_size, shuffle=False):
    ds = tf.data.Dataset.from_tensor_slices((texts, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(texts), seed=SEED, reshuffle_each_iteration=True)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds_raw = make_ds(X_train, y_train, BATCH_SIZE, shuffle=True)
val_ds_raw   = make_ds(X_val,   y_val,   BATCH_SIZE, shuffle=False)

# Mapea texto -> ids con la capa de vectorización dentro del pipeline
def map_to_vector(x, y):
    return vectorizer(x), y

train_ds = train_ds_raw.map(map_to_vector, num_parallel_calls=tf.data.AUTOTUNE)
val_ds   = val_ds_raw.map(map_to_vector,   num_parallel_calls=tf.data.AUTOTUNE)

# ---------- Modelo BiLSTM ----------
from tensorflow.keras import layers, models

inputs = layers.Input(shape=(None,), dtype=tf.int32, name="token_ids")
x = layers.Embedding(input_dim=MAX_TOKENS, output_dim=EMB_DIM, mask_zero=True, name="embedding")(inputs)
x = layers.Bidirectional(layers.LSTM(LSTM_UNITS, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(LSTM_UNITS//2))(x)
x = layers.Dropout(DROPOUT)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(DROPOUT)(x)

if is_binary:
    outputs = layers.Dense(1, activation="sigmoid")(x)
    loss = "binary_crossentropy"
    metrics = ["accuracy"]
else:
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    loss = "sparse_categorical_crossentropy"
    metrics = ["accuracy"]

model = models.Model(inputs, outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR), loss=loss, metrics=metrics)
model.summary()

# ---------- Callbacks ----------
ckpt_dir = os.path.join(SAVE_BASE, "checkpoints_lstm")
os.makedirs(ckpt_dir, exist_ok=True)
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=2, mode="max", restore_best_weights=True),
]

# ---------- Entrenamiento ----------
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

# ---------- Evaluación detallada ----------
# Predicciones en valid
y_pred_prob = model.predict(val_ds, verbose=0)
if is_binary:
    y_pred = (y_pred_prob.ravel() >= 0.5).astype(int)
    target_names = ["not_offensive", "offensive"]
else:
    y_pred = np.argmax(y_pred_prob, axis=1)
    target_names = list(le.classes_)

print("\n== Métricas de validación ==")
acc = accuracy_score(y_val, y_pred)
pr, rc, f1, _ = precision_recall_fscore_support(y_val, y_pred, average="weighted", zero_division=0)
print(f"accuracy: {acc:.4f} | f1_weighted: {f1:.4f} | precision_weighted: {pr:.4f} | recall_weighted: {rc:.4f}")

print("\n== Classification report ==")
print(classification_report(y_val, y_pred, target_names=target_names, digits=4))
print("\n== Matriz de confusión ==")
print(confusion_matrix(y_val, y_pred))

# ---------- Guardar modelo + vectorizador + (label encoder) ----------
save_dir = os.path.join(SAVE_BASE, "lstm_model_saved")
os.makedirs(save_dir, exist_ok=True)

# Guardar el modelo Keras
model.save(os.path.join(save_dir, "model.keras"))

# Guardar el vectorizador como un modelo Keras aparte
vec_in = tf.keras.Input(shape=(1,), dtype=tf.string)
vec_out = vectorizer(vec_in)
vec_model = tf.keras.Model(vec_in, vec_out)
vec_model.save(os.path.join(save_dir, "vectorizer.keras"))

# Guardar mapa de etiquetas si es multiclase
if not is_binary:
    import joblib
    joblib.dump({"classes_": target_names}, os.path.join(save_dir, "emotion_label_map.joblib"))

print(f"\n📁 Guardado en: {save_dir}")

# ---------- Inferencia rápida ----------
def predict_texts(texts):
    # vectoriza
    x = vectorizer(tf.constant(texts))
    probs = model.predict(x, verbose=0)
    if is_binary:
        pred_ids = (probs.ravel() >= 0.5).astype(int)
        labels = ["not_offensive" if i==0 else "offensive" for i in pred_ids]
    else:
        pred_ids = np.argmax(probs, axis=1)
        labels = [target_names[i] for i in pred_ids]
    return [{"text": t, "label": l, "probs": (probs[i].tolist() if not is_binary else float(probs[i]))} for i, (t,l) in enumerate(zip(texts, labels))]

print("\n== Ejemplos de predicción ==")
for r in predict_texts([
    "Acabo de ver la gran pérdida que estamos teniendo y me siento devastado.",
    "Qué rabia lo que pasó ayer.",
    "Me da asco leer esos comentarios."
]):
    print(r)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 892.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 94.3 MB/s eta 0:00:00


## Modelo 2: Fine-tuning con BERT (Transformers de HuggingFace)

In [ ]:
# ============================================
# Fine-tuning rápido (5–10 min) con BERT + Trainer (PyTorch) - Compatible con versiones antiguas de transformers
# Dataset esperado: df[id, tweet, emotion, offensive]
# ============================================

!pip -q install -U "transformers" "datasets" "accelerate" \
                 "scikit-learn" "pandas" "numpy" "torch"

import os, random, numpy as np, pandas as pd, torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report

from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    set_seed
)

print("PyTorch:", torch.__version__)
import transformers, datasets
print("Transformers:", transformers.__version__)
print("Datasets:", datasets.__version__)

# Monta Google Drive si quieres guardar el modelo
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    SAVE_BASE = "/content/drive/MyDrive/bert_rapido_colab"
except:
    SAVE_BASE = "/content/bert_rapido_colab"
os.makedirs(SAVE_BASE, exist_ok=True)

# Configuración y semilla
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); set_seed(SEED)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Dispositivo:", device)

# --------- Carga tu dataset ----------
# df = pd.read_csv("/content/drive/MyDrive/tu_dataset.csv")
df = pd.DataFrame({
    "id": [1,2,3,4,5]*300,
    "tweet": [
        "Acabo de ver la gran pérdida que estamos teniendo...",
        "USER ¿A que vamos a reconstruir Notre Dame antes que...",
        "Desde ayer andan sufriendo por el incendio de ...",
        "Muy afectada por lo ocurrido...",
        "Es una mierda lo que pasó en HASHTAG..."
    ]*300,
    "emotion": ["sadness","sadness","anger","sadness","disgust"]*300,
    "offensive": [0,0,0,0,1]*300
})

# Validación de columnas
required = {"id","tweet","emotion","offensive"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"Faltan columnas: {missing}")
df = df.dropna(subset=["tweet"]).reset_index(drop=True)
df["tweet"] = df["tweet"].astype(str).str.strip()

# --------- Parámetros rápidos ----------
TARGET_COL  = "emotion"   # o "offensive"
TEXT_COL    = "tweet"
MODEL_NAME  = "dccuchile/bert-base-spanish-wwm-cased"

N_SUBSAMPLE = 1000
MAX_LEN     = 64
NUM_EPOCHS  = 1
BATCH_SIZE  = 32
LR          = 2e-5

if len(df) > N_SUBSAMPLE:
    df = df.sample(n=N_SUBSAMPLE, random_state=SEED).reset_index(drop=True)

# Etiquetas
if TARGET_COL == "emotion":
    le = LabelEncoder()
    df["labels"] = le.fit_transform(df[TARGET_COL])
    id2label = {i: lab for i, lab in enumerate(le.classes_)}
    label2id = {lab: i for i, lab in id2label.items()}
    num_labels = len(id2label)
else:
    df["labels"] = df[TARGET_COL].astype(int)
    id2label = {0: "not_offensive", 1: "offensive"}
    label2id = {"not_offensive": 0, "offensive": 1}
    num_labels = 2

# Split
train_df, val_df = train_test_split(
    df, test_size=0.15, random_state=SEED, stratify=df["labels"]
)

# HF Datasets
cols = [TEXT_COL, "labels"]
train_ds = Dataset.from_pandas(train_df[cols].reset_index(drop=True))
val_ds   = Dataset.from_pandas(val_df[cols].reset_index(drop=True))
dataset  = DatasetDict({"train": train_ds, "validation": val_ds})

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
def tokenize_fn(batch):
    return tokenizer(batch[TEXT_COL], truncation=True, padding="max_length", max_length=MAX_LEN)
tokenized = dataset.map(tokenize_fn, batched=True, remove_columns=[TEXT_COL])
tokenized.set_format(type="torch")

# Modelo
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
).to(device)

# TrainingArguments (versión antigua/compatible)
out_dir = os.path.join(SAVE_BASE, "runs_fast_compat")
args = TrainingArguments(
    output_dir=out_dir,
    do_eval=True,                 # permite .evaluate()
    logging_steps=20,
    save_steps=200,
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    seed=SEED
)

# Métricas
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    pr, rc, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=0)
    return {"accuracy": acc, "f1_weighted": f1, "precision_weighted": pr, "recall_weighted": rc}

# Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Entrenar
trainer.train()
print("\n✅ Entrenamiento terminado.")

# Evaluación
eval_results = trainer.evaluate()
print("\n== Métricas de validación ==")
print(eval_results)

preds_output = trainer.predict(tokenized["validation"])
logits = preds_output[0] if isinstance(preds_output, tuple) else preds_output.predictions
y_true = preds_output[1] if isinstance(preds_output, tuple) else preds_output.label_ids
y_pred = np.argmax(logits, axis=-1)

target_names = [id2label[i] for i in range(num_labels)] if TARGET_COL=="emotion" else ["not_offensive","offensive"]
print("\n== Classification report ==")
print(classification_report(y_true, y_pred, target_names=target_names, digits=4))
print("\n== Matriz de confusión ==")
print(confusion_matrix(y_true, y_pred))

# Guardar
save_dir = os.path.join(SAVE_BASE, "model_fast_compat")
os.makedirs(save_dir, exist_ok=True)
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"\nModelo guardado en: {save_dir}")

# Inferencia rápida
def predict_texts(texts):
    enc = tokenizer(texts, truncation=True, padding="max_length", max_length=MAX_LEN, return_tensors="pt").to(device)
    model.eval()
    with torch.no_grad():
        out = model(**enc)
        probs = torch.softmax(out.logits, dim=-1).cpu().numpy()
        preds = probs.argmax(axis=-1)
    labels = [id2label[i] for i in preds]
    return [{"text": t, "label": l, "probs": p.tolist()} for t, l, p in zip(texts, labels, probs)]

print("\n== Ejemplo de predicción ==")
print(predict_texts([
    "Acabo de ver la gran pérdida que estamos teniendo y me siento devastado.",
    "Qué rabia lo que pasó ayer.",
    "Me da asco leer esos comentarios."
]))

PyTorch: 2.8.0+cu126
Transformers: 4.55.2
Datasets: 4.0.0
Mounted at /content/drive
Dispositivo: cpu


Map:   0%|          | 0/850 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-4260555667.py:141: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
20,0.319200



✅ Entrenamiento terminado.


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



== Métricas de validación ==
{'eval_loss': 0.021178748458623886, 'eval_accuracy': 1.0, 'eval_f1_weighted': 1.0, 'eval_precision_weighted': 1.0, 'eval_recall_weighted': 1.0, 'eval_runtime': 36.6012, 'eval_samples_per_second': 4.098, 'eval_steps_per_second': 0.137, 'epoch': 1.0}


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



== Classification report ==
              precision    recall  f1-score   support

       anger     1.0000    1.0000    1.0000        30
     disgust     1.0000    1.0000    1.0000        30
     sadness     1.0000    1.0000    1.0000        90

    accuracy                         1.0000       150
   macro avg     1.0000    1.0000    1.0000       150
weighted avg     1.0000    1.0000    1.0000       150


== Matriz de confusión ==
[[30  0  0]
 [ 0 30  0]
 [ 0  0 90]]

Modelo guardado en: /content/drive/MyDrive/bert_rapido_colab/model_fast_compat

== Ejemplo de predicción ==
[{'text': 'Acabo de ver la gran pérdida que estamos teniendo y me siento devastado.', 'label': 'sadness', 'probs': [0.01624990813434124, 0.02740885317325592, 0.9563412070274353]}, {'text': 'Qué rabia lo que pasó ayer.', 'label': 'sadness', 'probs': [0.09263766556978226, 0.1568194180727005, 0.750542938709259]}, {'text': 'Me da asco leer esos comentarios.', 'label': 'sadness', 'probs': [0.13736765086650848, 0.238276

## Modelo 3: Uso de un Modelo Preentrenado Grande (GPT-like)

In [ ]:
# ============================================
# Modelo 3 — GPT-like (GPT-2 español) para CLASIFICACIÓN con Trainer (PyTorch)
# Compatible con versiones antiguas de transformers (sin evaluation_strategy)
# Dataset esperado: df[id, tweet, emotion, offensive]
# Objetivo por defecto: 'emotion' (multiclase). Cambia TARGET_COL a 'offensive' (binario).
# ============================================

# ---------- Instalar deps ----------
!pip -q install -U "transformers" "datasets" "accelerate" \
                 "scikit-learn" "pandas" "numpy" "torch"

# ---------- Imports y setup ----------
import os, random, numpy as np, pandas as pd, torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report

from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    set_seed
)

print("PyTorch:", torch.__version__)
import transformers, datasets
print("Transformers:", transformers.__version__)
print("Datasets:", datasets.__version__)

# Montar Google Drive (opcional)
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    SAVE_BASE = "/content/drive/MyDrive/gpt_like_clf_colab"
except:
    SAVE_BASE = "/content/gpt_like_clf_colab"
os.makedirs(SAVE_BASE, exist_ok=True)

# Semillas y dispositivo
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); set_seed(SEED)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Dispositivo:", device)

# ---------- CARGA TU DATA ----------
# Reemplaza por tu CSV si ya lo tienes:
# df = pd.read_csv("/content/drive/MyDrive/tu_dataset.csv")

# Ejemplo mínimo (borra si usas CSV)
df = pd.DataFrame({
    "id": [1,2,3,4,5]*300,  # simula 1500 filas
    "tweet": [
        "Acabo de ver la gran pérdida que estamos teniendo...",
        "USER ¿A que vamos a reconstruir Notre Dame antes que...",
        "Desde ayer andan sufriendo por el incendio de ...",
        "Muy afectada por lo ocurrido...",
        "Es una mierda lo que pasó en HASHTAG..."
    ]*300,
    "emotion": ["sadness","sadness","anger","sadness","disgust"]*300,
    "offensive": [0,0,0,0,1]*300
})

required = {"id","tweet","emotion","offensive"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"Faltan columnas en df: {missing}")
df = df.dropna(subset=["tweet"]).reset_index(drop=True)
df["tweet"] = df["tweet"].astype(str).str.strip()

# ---------- Parámetros rápidos ----------
TARGET_COL  = "emotion"      # "emotion" (multiclase) o "offensive" (binario)
TEXT_COL    = "tweet"
# Modelos GPT-2 en ES recomendados (elige uno):
MODEL_NAME  = "datificate/gpt2-small-spanish"     # pequeño y rápido
# MODEL_NAME  = "PlanTL-GOB-ES/gpt2-base-bne"     # más grande (tardará más)

N_SUBSAMPLE = 1000    # submuestreo para 5–10 min
MAX_LEN     = 64      # secuencia corta
NUM_EPOCHS  = 1
BATCH_SIZE  = 32      # si hay OOM baja a 16/8
LR          = 2e-5

# ---------- Submuestreo ----------
if len(df) > N_SUBSAMPLE:
    df = df.sample(n=N_SUBSAMPLE, random_state=SEED).reset_index(drop=True)

# ---------- Etiquetas ----------
if TARGET_COL == "emotion":
    le = LabelEncoder()
    df["labels"] = le.fit_transform(df[TARGET_COL])
    id2label = {i: lab for i, lab in enumerate(le.classes_)}
    label2id = {lab: i for i, lab in id2label.items()}
    num_labels = len(id2label)
else:
    df["labels"] = df[TARGET_COL].astype(int)
    id2label = {0: "not_offensive", 1: "offensive"}
    label2id = {"not_offensive": 0, "offensive": 1}
    num_labels = 2

# ---------- Split ----------
train_df, val_df = train_test_split(
    df, test_size=0.15, random_state=SEED, stratify=df["labels"]
)

# ---------- Datasets HF ----------
cols = [TEXT_COL, "labels"]
train_ds = Dataset.from_pandas(train_df[cols].reset_index(drop=True))
val_ds   = Dataset.from_pandas(val_df[cols].reset_index(drop=True))
dataset  = DatasetDict({"train": train_ds, "validation": val_ds})

# ---------- Tokenizer ----------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
# GPT-2 no tiene pad_token por defecto; igualamos PAD=EOS para poder hacer padding
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_fn(batch):
    return tokenizer(
        batch[TEXT_COL],
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN
    )

tokenized = dataset.map(tokenize_fn, batched=True, remove_columns=[TEXT_COL])
tokenized.set_format(type="torch")

# ---------- Modelo (GPT-like con cabeza de clasificación) ----------
# Usamos AutoModelForSequenceClassification para añadir la cabeza lineal de clasificación
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)
# Alinear pad_token_id con el tokenizer para evitar warnings en pérdida por padding
model.config.pad_token_id = tokenizer.pad_token_id
model.to(device)

# ---------- TrainingArguments (compatibles) ----------
out_dir = os.path.join(SAVE_BASE, "runs_gpt_like_fast_compat")
args = TrainingArguments(
    output_dir=out_dir,
    do_eval=True,                 # permite .evaluate()
    logging_steps=20,
    save_steps=200,               # checkpoints periódicos (puedes subir/bajar)
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    seed=SEED
    # NO usamos evaluation_strategy / save_strategy para compatibilidad
)

# ---------- Métricas ----------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    pr, rc, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=0)
    return {"accuracy": acc, "f1_weighted": f1, "precision_weighted": pr, "recall_weighted": rc}

# ---------- Trainer ----------
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# ---------- Entrenamiento ----------
trainer.train()
print("\n✅ Entrenamiento terminado.")

# ---------- Evaluación ----------
eval_results = trainer.evaluate()
print("\n== Métricas de validación ==")
print(eval_results)

preds_output = trainer.predict(tokenized["validation"])
logits = preds_output[0] if isinstance(preds_output, tuple) else preds_output.predictions
y_true = preds_output[1] if isinstance(preds_output, tuple) else preds_output.label_ids
y_pred = np.argmax(logits, axis=-1)

target_names = [id2label[i] for i in range(len(id2label))] if TARGET_COL=="emotion" else ["not_offensive","offensive"]
print("\n== Classification report ==")
print(classification_report(y_true, y_pred, target_names=target_names, digits=4))
print("\n== Matriz de confusión ==")
print(confusion_matrix(y_true, y_pred))

# ---------- Guardar ----------
save_dir = os.path.join(SAVE_BASE, "gpt_like_model_fast_compat")
os.makedirs(save_dir, exist_ok=True)
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"\nModelo y tokenizer guardados en: {save_dir}")

# ---------- Inferencia rápida ----------
def predict_texts(texts):
    enc = tokenizer(texts, truncation=True, padding="max_length", max_length=MAX_LEN, return_tensors="pt").to(device)
    model.eval()
    with torch.no_grad():
        out = model(**enc)
        probs = torch.softmax(out.logits, dim=-1).cpu().numpy()
        preds = probs.argmax(axis=-1)
    labels = [id2label[i] for i in preds]
    return [{"text": t, "label": l, "probs": p.tolist()} for t, l, p in zip(texts, labels, probs)]

print("\n== Ejemplo de predicción ==")
print(predict_texts([
    "Acabo de ver la gran pérdida que estamos teniendo y me siento devastado.",
    "Qué rabia lo que pasó ayer.",
    "Me da asco leer esos comentarios."
]))

PyTorch: 2.8.0+cu126
Transformers: 4.55.2
Datasets: 4.0.0
Mounted at /content/drive
Dispositivo: cpu


tokenizer_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...


✅ Entrenamiento terminado.


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



== Métricas de validación ==
{'eval_loss': 0.0009275110787712038, 'eval_accuracy': 1.0, 'eval_f1_weighted': 1.0, 'eval_precision_weighted': 1.0, 'eval_recall_weighted': 1.0, 'eval_runtime': 38.6604, 'eval_samples_per_second': 3.88, 'eval_steps_per_second': 0.129, 'epoch': 1.0}


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



== Classification report ==
              precision    recall  f1-score   support

       anger     1.0000    1.0000    1.0000        30
     disgust     1.0000    1.0000    1.0000        30
     sadness     1.0000    1.0000    1.0000        90

    accuracy                         1.0000       150
   macro avg     1.0000    1.0000    1.0000       150
weighted avg     1.0000    1.0000    1.0000       150


== Matriz de confusión ==
[[30  0  0]
 [ 0 30  0]
 [ 0  0 90]]

Modelo y tokenizer guardados en: /content/drive/MyDrive/gpt_like_clf_colab/gpt_like_model_fast_compat

== Ejemplo de predicción ==
[{'text': 'Acabo de ver la gran pérdida que estamos teniendo y me siento devastado.', 'label': 'sadness', 'probs': [0.04312266781926155, 0.033531513065099716, 0.9233458638191223]}, {'text': 'Qué rabia lo que pasó ayer.', 'label': 'sadness', 'probs': [0.09839481860399246, 0.07850045710802078, 0.8231046795845032]}, {'text': 'Me da asco leer esos comentarios.', 'label': 'sadness', 'probs': [0.0